# Train

1. Load the Iris dataset from sk-learn (150x4 numpy.ndarray)
2. Cluster the data by using k-means algorithm
3. Save the trained model in onnx file

In [6]:
model_filename = "rf_iris.onnx"

In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from skl2onnx import to_onnx
from sklearn.cluster import KMeans


print("Loading data")
iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)


X_train, X_test, y_train, y_test = train_test_split(X, y)

print("Train kMeans...")
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10,
                random_state=0)
y_kmeans = kmeans.fit_predict(X_train, y_train)

# Convert into ONNX format.
onx = to_onnx(kmeans, X[:1])
with open(model_filename, "wb") as f:
    f.write(onx.SerializeToString())
    
print(f"Model saved as {model_filename}")

Loading data
Train kMeans...
Model saved as rf_iris.onnx


# Upload Model

In [4]:
! pip install boto3


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [8]:
import os
import boto3

s3_destination_path = "models/model.onnx"

key_id = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
endpoint = os.getenv("AWS_S3_ENDPOINT")
bucket_name = os.getenv("AWS_S3_BUCKET")

s3 = boto3.client(
   "s3",
   aws_access_key_id=key_id,
   aws_secret_access_key=secret_key,
   endpoint_url=endpoint,
   use_ssl=True)

s3.upload_file(model_filename, bucket_name, Key=s3_destination_path)

print(f"Uploaded model to bucket {bucket_name}")

Uploaded model to bucket mybucket


# Use model 

After having created a model serving runtime with a deployed model, you can use the model server to inference data

In [21]:
model_server_endpoint = "REPLACE_ME"
token = "REPLACE_ME"

In [ ]:
import requests


headers = {
    'Authorization': f'Bearer {token}'
}
payload = {"inputs" : [{"name" : "X","shape" : [ 1, 4 ],"datatype" : "FP32","data" : [ 3, 4, 3, 2 ]}],"outputs" : [{"name" : "output0"}]}

response = requests.post(
    model_server_endpoint, 
    headers = headers,
    json = payload
)

if response.ok:
    print(response.json())
else:
    raise Exception(response.text)